Creating a clean plate

Zadanie polega wygenerowaniu obrazu katedry bez przechodzących ludzi na podstawie wielu obrazów wejściowych. 

Do wykonania tego zadania zostały wykorzystane takie operacje na obrazach jak wyliczenie histogramu, utworzenie i nałożenie maski, znalezienie najczęściej występujących pixeli. Dodatkowo została utworzona funkcja obliczająca dylatacje dla wybranych pixeli z obrazu. 

Lista kroków:

1) Załaduj wszystkie obrazy
2) Znajdź najczęściej występujące pixele w obrazach i utwórz z nich obraz pomocniczy 'combined_image'
3) Wyznacz histogram z 'combined_image'
4) Znajdź wartości szczytowe histogramu, które są większe od wartości maksymalnej histogramu, przeskalowanej o współczynnik skalujący
5) Utwórz maskę na podstawie wartości szczytowych histogramu
6) Utwórz obraz 'masked_image' na podstawie nałożenia maski na 'combined_image'
7) Przeprowadź operacje dylatacji dla czarnych pixeli za pomocą funkcji custom_median_dilate() i zapisz go jako 'masked_image_dil_mean'
8) Połącz drugi obraz wejściowy z obrazem 'result_image_dil_mean' i zapisz jako 'final_result'

In [27]:
# import bibliotek
import cv2
import numpy as np

In [28]:
def custom_median_dilate(image, kernel_size, tolerance=30):
    """
    Funkcja do obliczania dylatacji na podstawie obrazu wejściowego.
    Dla każdego czarnego pixela w pewnym zakresie tolerancji wyznacza średnią na podstawie wszystkich nieczarnych pixeli w swoim otoczeniu.
    """
    height, width, _ = image.shape
    result = np.copy(image) 

    for y in range(height):
        for x in range(width):
            if np.all(np.abs(image[y, x] - [0, 0, 0]) <= tolerance):  # sprawdź czy pixel jest w zakresie tolerancji
                neighborhood = image[max(0, y - kernel_size):min(height, y + kernel_size + 1),
                                        max(0, x - kernel_size):min(width, x + kernel_size + 1)]
                non_black_pixels = neighborhood[~np.all(np.abs(neighborhood - [0, 0, 0]) <= tolerance, axis=-1)] #sprawdz które pixele w sasiedztwie są nieczarne
                if non_black_pixels.size > 0:
                    result[y, x] = np.median(non_black_pixels, axis=0).astype(np.uint8) #wyznacz mediane ze wszystkich nieczarnych pixeli
    image[:] = result 

    return result

In [31]:
# Ładowanie wszystkich 5 obrazów i usuwanie ludzi
images = [cv2.imread(f"cathedral/d00{i}.jpg") for i in range(1, 6)]

# Obliczanie współrzędnych punktów, w których obrazy zostaną nakładane na siebie
height, width, _ = images[0].shape
combined_image = np.zeros((height, width, 3), dtype=np.uint8)
kernel = np.ones((4, 4), np.uint8) 


# Znajdowanie najczęstszego piksela dla każdego piksela pomiędzy obrazami oraz utworzenie obrazu z najczęściej występujących pixeli
for y in range(height):
    for x in range(width):
        pixel_values = [tuple(img[y, x]) for img in images]
        most_frequent_value = max(set(pixel_values), key=pixel_values.count)
        combined_image[y, x] = most_frequent_value

cv2.imwrite("combined_image.jpg", combined_image)

# Wyznaczenie histogramu utworzonego zdjęcia
histogram = cv2.calcHist([combined_image], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])

# Znaleźć szczyty histogramu
threshold=0.007
peaks = np.where(histogram > np.max(histogram) * threshold) #Mozna zmieniac liczbe przez ktora mnoże się max, dobre sa z zakresu 0.01 - 0.004

# Utworzenie maski na podstawie szczytów histogramu
mask = np.zeros((height, width), dtype=np.uint8)
for i in range(len(peaks[0])):
    mask[np.where((combined_image[:, :, 0] == peaks[0][i]) & (combined_image[:, :, 1] == peaks[1][i]) & (combined_image[:, :, 2] == peaks[2][i]))] = 255

# Nałożenie maski w celu wykrycia pozostałych pixeli ze złączonego zdjęcia
masked_image = cv2.bitwise_and(combined_image, combined_image, mask=mask)

cv2.imwrite("result_image.jpg", masked_image)
# Wykorzystanie własnej funkcji dylatacji
custom_kernel_size = 4
tolerance = 35
#najlepiej dla tolerance 35, kernel 4
masked_image_dil_mean = custom_median_dilate(masked_image, custom_kernel_size, tolerance= tolerance)
cv2.imwrite("result_image_dil_mean.jpg", masked_image_dil_mean)

# Połączenie górnej części zdjęcia katedry bez ludzi z dolną częścią na której dokonano dylatacji
split=350
final_result = np.concatenate((images[1][:split,:,:], masked_image_dil_mean[split:,:,:]), axis=0)

cv2.imwrite("final_result.jpg", final_result)

True